In [1]:
pip install plotly

In [5]:
pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.5 MB/s eta 0:00:00


In [6]:
import pandas as pd
import re
import openai
import time

import openai
openai.api_key

In [153]:
import openai
import re
import time
import plotly.graph_objects as go
import textwrap  # 텍스트를 여러 줄로 나누기 위해 사용

def evaluate_question(question):
    max_tries = 3  # 최대 재시도 횟수
    evaluations = {
        '관련성': 0.0,
        '명확성': 0.0,
        '개방성': 0.0,
        '공정성': 0.0,
        '유용성': 0.0
    }

    for eval_type in evaluations:
        try_count = 0
        while try_count < max_tries:
            try:
                prompt = (
                    f"평가 기준: {eval_type}\n"
                    f"질문: {question}\n"
                    "다음과 같은 인터뷰 상황을 고려하세요:\n"
                    "면접관이 후보자의 적합성을 평가하기 위해 질문을 하고 있습니다. "
                )
                if eval_type == '관련성':
                    prompt += "이 질문이 직무와 얼마나 관련이 있는지를 평가하세요. "
                elif eval_type == '명확성':
                    prompt += "이 질문이 얼마나 이해하기 쉽고 명확한지를 평가하세요. "
                elif eval_type == '개방성':
                    prompt += "이 질문이 면접자에게 다양한 대답을 유도할 가능성이 얼마나 있는지를 평가하세요. "
                elif eval_type == '공정성':
                    prompt += "이 질문이 편견이 없고 차별적이지 않은지 여부를 평가하세요. "
                elif eval_type == '유용성':
                    prompt += "이 질문이 후보자의 능력, 자격, 적합성에 대한 통찰을 제공하는지 여부를 평가하세요. "

                prompt += "0.00에서 10.00까지의 범위에서 평가해 주세요. 0.00은 전혀 해당 사항 없음, 10.00은 매우 해당됩니다."

                response = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=[{"role": "user", "content": prompt}])
                match = re.search(r'\d+\.\d+', response.choices[0].message.content)
                if match:
                    score = float(match.group())
                else:
                    print(f"No valid score found in response for {eval_type}. Assuming a neutral score of 5.00.")
                    score = 5.00
                evaluations[eval_type] = score
                break

            except openai.error.APIError as e:
                print(f"{eval_type} 평가에서 API 오류 발생, 재시도 중... ({try_count + 1}번째 시도)")
                time.sleep(2)
                try_count += 1
                if try_count >= max_tries:
                    print(f"{eval_type} 평가 실패: 최대 재시도 횟수 도달, 점수 기록 없음")
                    evaluations[eval_type] = 5.00

    return evaluations

def plot_radar_chart_with_plotly(evaluations, question):
    categories = list(evaluations.keys())
    values = list(evaluations.values())

    # Close the radar chart
    categories += [categories[0]]
    values += [values[0]]

    fig = go.Figure()
    fig.add_trace(go.Scatterpolar(
        r=values,
        theta=categories,
        fill='toself',
        name='질문 평가'
    ))

    # Calculate average score
    average_score = sum(values[:-1]) / len(values[:-1])
    average_score = round(average_score, 2)

    wrapped_question = "<br>".join(textwrap.wrap(question, width=80))

    # Title setup to include question
    title_text = f"질문 평가 레이더 차트<br><sub>{wrapped_question}</sub><br><sub>Average Score: {average_score:.2f}</sub>"

    fig.update_layout(
        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 10]
            )
        ),
        title=title_text,
        title_x=0.05,
        title_y=0.95,
        width=800,
        height=600,
        margin=dict(l=50, r=50, t=100, b=50),
        font=dict(family="Malgun Gothic", size=14, color="black")
    )

    fig.show()


In [149]:
question = "최근 팀 프로젝트에서 어떻게 리더십을 발휘했나요?"
evaluations = evaluate_question(question)
plot_radar_chart_with_plotly(evaluations, question)

In [154]:
question = "지원자님께서 지금부터 삼 년 뒤까지 삼 년 내에 본인이 꼭 이루고 싶은 인생의 목표가 있으신지 궁금합니다 있으시다면 그것이 무엇인지 말씀해 주시기 바랍니다."
evaluations = evaluate_question(question)
plot_radar_chart_with_plotly(evaluations, question)

In [155]:
question = "십 년 후에 제가 생각하는 저의 미래는 어떨 것 같으신가요 어떤 자신감 있는 외모와 어떤 자신감 있는 외모를 가진지 귀여운 아이를 낳는지는 모르겠습니다만 훌륭한 부모를 맡아보는 것 같습니다."
evaluations = evaluate_question(question)
plot_radar_chart_with_plotly(evaluations, question)

In [152]:
question = "지원자님 주변에 많은 친구들이 있을 텐데서 이제 저희와 가장 가까운 친구는 어떤 사람일까요?"
evaluations = evaluate_question(question)
plot_radar_chart_with_plotly(evaluations, question)